In [1]:
pip install torch --upgrade


Note: you may need to restart the kernel to use updated packages.


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Define number of classes
num_classes = 2  # Replace with your actual number of classes

import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define your transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(40),
        transforms.ColorJitter(brightness=0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Set the data directory path
data_dir = '/kaggle/input/indian-currency-dataset/Indian Currency Dataset'

# Create datasets using ImageFolder
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'test']}

# Create DataLoader for each dataset
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4) for x in ['train', 'test']}


# Use CUDA if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load and modify the MobileNetV2 model
mobilenet_model = models.mobilenet_v2(pretrained=True)
mobilenet_model.classifier[1] = nn.Linear(mobilenet_model.last_channel, num_classes)
mobilenet_model = mobilenet_model.to(device)

# Load and modify the EfficientNetB0 model
efficientnet_model = models.efficientnet_b0(pretrained=True)
efficientnet_model.classifier[1] = nn.Linear(efficientnet_model.classifier[1].in_features, num_classes)
efficientnet_model = efficientnet_model.to(device)

# Training function
def train_model(model, criterion, optimizer, num_epochs=25):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    return model

# Set up criterion and optimizers
criterion = nn.CrossEntropyLoss()
optimizer_mobilenet = optim.Adam(mobilenet_model.parameters(), lr=0.001)
optimizer_efficientnet = optim.Adam(efficientnet_model.parameters(), lr=0.001)

# Train the models
mobilenet_model = train_model(mobilenet_model, criterion, optimizer_mobilenet, num_epochs=25)
efficientnet_model = train_model(efficientnet_model, criterion, optimizer_efficientnet, num_epochs=25)

# Feature extraction function
def extract_features(model, dataloader):
    model.eval()
    features = []
    labels = []

    with torch.no_grad():
        for inputs, labels_batch in dataloader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            features.append(outputs.cpu().numpy())
            labels.append(labels_batch.numpy())

    return np.concatenate(features), np.concatenate(labels)

# Extract features using both models
mobilenet_features_train, train_labels = extract_features(mobilenet_model, dataloaders['train'])
mobilenet_features_val, val_labels = extract_features(mobilenet_model, dataloaders['test'])
efficientnet_features_train, _ = extract_features(efficientnet_model, dataloaders['train'])
efficientnet_features_val, _ = extract_features(efficientnet_model, dataloaders['test'])

# Stack the features
train_features = np.hstack((mobilenet_features_train, efficientnet_features_train))
val_features = np.hstack((mobilenet_features_val, efficientnet_features_val))

# Define classifiers
rf = RandomForestClassifier(n_estimators=100, random_state=42)
gb = GradientBoostingClassifier(n_estimators=100, random_state=42)
lr = LogisticRegression(max_iter=1000)
svc = SVC(probability=True)

# Train the classifiers
rf.fit(train_features, train_labels)
gb.fit(train_features, train_labels)
lr.fit(train_features, train_labels)
svc.fit(train_features, train_labels)

# Validate the classifiers
rf_preds = rf.predict(val_features)
gb_preds = gb.predict(val_features)
lr_preds = lr.predict(val_features)
svc_preds = svc.predict(val_features)

# Print accuracy scores
print("Random Forest Accuracy:", accuracy_score(val_labels, rf_preds))
print("Gradient Boosting Accuracy:", accuracy_score(val_labels, gb_preds))
print("Logistic Regression Accuracy:", accuracy_score(val_labels, lr_preds))
print("SVC Accuracy:", accuracy_score(val_labels, svc_preds))


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get th

Epoch 1/25
----------
train Loss: 0.7132 Acc: 0.6210
test Loss: 0.6132 Acc: 0.5981
Epoch 2/25
----------
train Loss: 0.4777 Acc: 0.7863
test Loss: 0.9016 Acc: 0.7570
Epoch 3/25
----------
train Loss: 0.4359 Acc: 0.7823
test Loss: 0.4527 Acc: 0.7477
Epoch 4/25
----------
train Loss: 0.4017 Acc: 0.8266
test Loss: 0.8442 Acc: 0.7850
Epoch 5/25
----------
train Loss: 0.5145 Acc: 0.7903
test Loss: 0.4640 Acc: 0.7383
Epoch 6/25
----------
train Loss: 0.4389 Acc: 0.7944
test Loss: 0.3257 Acc: 0.8692
Epoch 7/25
----------
train Loss: 0.3898 Acc: 0.8347
test Loss: 0.7360 Acc: 0.8131
Epoch 8/25
----------
train Loss: 0.3363 Acc: 0.8871
test Loss: 0.2776 Acc: 0.8785
Epoch 9/25
----------
train Loss: 0.4046 Acc: 0.8347
test Loss: 0.3715 Acc: 0.8224
Epoch 10/25
----------
train Loss: 0.3842 Acc: 0.8427
test Loss: 0.2594 Acc: 0.8972
Epoch 11/25
----------
train Loss: 0.3344 Acc: 0.8669
test Loss: 0.2491 Acc: 0.8598
Epoch 12/25
----------
train Loss: 0.3518 Acc: 0.8387
test Loss: 0.2084 Acc: 0.9252
E

KeyError: 'val'

In [5]:
# Extract features using both models
mobilenet_features_train, train_labels = extract_features(mobilenet_model, dataloaders['train'])
mobilenet_features_val, val_labels = extract_features(mobilenet_model, dataloaders['test'])
efficientnet_features_train, _ = extract_features(efficientnet_model, dataloaders['train'])
efficientnet_features_val, _ = extract_features(efficientnet_model, dataloaders['test'])

# Stack the features
train_features = np.hstack((mobilenet_features_train, efficientnet_features_train))
val_features = np.hstack((mobilenet_features_val, efficientnet_features_val))

# Define classifiers
rf = RandomForestClassifier(n_estimators=100, random_state=42)
gb = GradientBoostingClassifier(n_estimators=100, random_state=42)
lr = LogisticRegression(max_iter=1000)
svc = SVC(probability=True)

# Train the classifiers
rf.fit(train_features, train_labels)
gb.fit(train_features, train_labels)
lr.fit(train_features, train_labels)
svc.fit(train_features, train_labels)

# Validate the classifiers
rf_preds = rf.predict(val_features)
gb_preds = gb.predict(val_features)
lr_preds = lr.predict(val_features)
svc_preds = svc.predict(val_features)

# Print accuracy scores
print("Random Forest Accuracy:", accuracy_score(val_labels, rf_preds))
print("Gradient Boosting Accuracy:", accuracy_score(val_labels, gb_preds))
print("Logistic Regression Accuracy:", accuracy_score(val_labels, lr_preds))
print("SVC Accuracy:", accuracy_score(val_labels, svc_preds))


Random Forest Accuracy: 0.9345794392523364
Gradient Boosting Accuracy: 0.9439252336448598
Logistic Regression Accuracy: 0.9345794392523364
SVC Accuracy: 0.9345794392523364
